In [1]:
from typing import TypedDict,Annotated
from dotenv import load_dotenv
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph,END,add_messages
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",
)
search_tool = TavilySearchResults(max_results=4)
llm_with_tools = llm.bind_tools([search_tool])

/var/folders/gk/cm66kf9j5qj4vzctc4gt_sxr0000gn/T/ipykernel_27346/1835026789.py:13: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=4)


In [2]:
class ChildState(TypedDict):
    messages: Annotated[list,add_messages]

In [3]:
def llm_node(state: ChildState):
    return {
        "messages":[llm_with_tools.invoke(state["messages"])]
    }
def conditional_edge(state: ChildState):
 return "tool_node" if hasattr(state["messages"][-1],"tool_calls") and len(state["messages"][-1].tool_calls)>0 else END

In [4]:
tool_node = ToolNode([search_tool],messages_key="messages")
graph = StateGraph(ChildState)
graph.add_node("llm_node",llm_node)
graph.add_node("tool_node",tool_node)
graph.add_conditional_edges("llm_node",conditional_edge,{"tool_node":"tool_node",END:END})
graph.add_edge("tool_node","llm_node")
graph.set_entry_point("llm_node")
app = graph.compile()